In [2]:
import copy, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

In [4]:
ncaab_2021_df = pd.read_csv('2021_team_results.csv')
ncaab_2022_df = pd.read_csv('2022_team_results.csv')
ncaab_2023_df = pd.read_csv('2023_team_results.csv')
ncaab_2024_df = pd.read_csv('2024_team_results.csv')

#test here
print(os.getcwd())
test_df = pd.read_csv("c:\Users\lakea\Desktop\Notes\OMO-ML\\nfl\\2021_team_results.csv")

c:\Users\lakea\Desktop\Notes\OMO-ML\ncaab


In [ ]:
test_df